In [7]:
from Tempo import Tempo
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
obj = Tempo()

In [8]:
end_date = date(2018, 8, 12)
start_date = date(2018, 1, 4)

In [9]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], single_date.strftime("%Y/%m/%d"))

2018-01-04
https://www.tempo.co/indeks/2018/01/04
https://nasional.tempo.co/read/1047487/kasus-blbi-eks-ketua-bppn-urusan-saya-sudah-selesai
memasukkan berita id  1047487
Insert berita  Kasus BLBI, Eks Ketua BPPN: Urusan Saya Sudah Selesai
masuk
https://pilkada.tempo.co/read/1047486/isu-azwar-anas-mundur-dari-pilgub-jatim-gus-ipul-saya-terkejut
memasukkan berita id  1047486
Insert berita  Isu Azwar Anas Mundur dari Pilgub Jatim, Gus Ipul: Saya Terkejut
masuk
https://bisnis.tempo.co/read/1047484/sandiaga-uno-proyek-lrt-jakarta-butuh-investasi-rp-336-t
memasukkan berita id  1047484
Insert berita  Sandiaga Uno: Proyek LRT Jakarta Butuh Investasi Rp 336 T  
masuk
https://pilkada.tempo.co/read/1047483/deklarasi-duo-dm-untuk-pilgub-jabar-dijadwalkan-9-januari
memasukkan berita id  1047483
Insert berita   Deklarasi Duo DM untuk Pilgub Jabar Dijadwalkan  9 Januari
masuk
https://nasional.tempo.co/read/1047482/dituduh-sebarkan-komunisme-aktivis-lingkungan-dituntut-7-tahun
memasukkan berita id  1

memasukkan berita id  1047430
Insert berita   Coutinho Ingin ke Barcelona, Neymar Coba Ajak ke PSG
masuk
https://metro.tempo.co/read/1047431/anies-baswedan-dan-sandiaga-uno-serahkan-jaket-bj-habibie-ke-kpk
memasukkan berita id  1047431
Insert berita  Anies Baswedan dan Sandiaga Uno Serahkan Jaket BJ Habibie ke KPK
masuk
https://bisnis.tempo.co/read/1047428/jalan-tol-ngawi-kertosono-segera-dioperasikan
memasukkan berita id  1047428
Insert berita  Jalan Tol Ngawi - Kertosono Segera Dioperasikan
masuk
https://nasional.tempo.co/read/1047427/akan-maju-pilkada-wali-kota-makassar-diperiksa-polisi
memasukkan berita id  1047427
Insert berita  Akan Maju Pilkada, Wali Kota Makassar Diperiksa Polisi
masuk
https://bola.tempo.co/read/1047426/copa-del-rey-hadapi-numancia-real-madrid-pasang-tim-lapis-kedua
memasukkan berita id  1047426
Insert berita  Copa del Rey: Hadapi Numancia, Real Madrid Pasang Tim Lapis Kedua
masuk
https://cantik.tempo.co/read/1047410/jangan-tiru-jennifer-dunn-pakai-narkoba-baha

masuk
https://gaya.tempo.co/read/1047377/ramalan-2018-ada-pelakor-dan-pebinor-alias-perebut-bini-orang
memasukkan berita id  1047377
Insert berita  Ramalan 2018: Ada Pelakor dan Pebinor alias Perebut Bini Orang
masuk
https://metro.tempo.co/read/1047376/ini-cara-pemprov-dki-bikin-blok-g-tanah-abang-ramai-pengunjung
memasukkan berita id  1047376
Insert berita  Ini Cara Pemprov DKI Bikin Blok G Tanah Abang Ramai Pengunjung  
masuk
https://cantik.tempo.co/read/1047375/ibu-hamil-kecanduan-narkoba-dampaknya-langsung-pada-janin
memasukkan berita id  1047375
Insert berita  Ibu Hamil Kecanduan Narkoba, Dampaknya Langsung pada Janin
masuk


KeyboardInterrupt: 